## OBJETIVO:
Identificar cada cicatriz de fogo nos mapas anuais do MapBiomas-Fogo sem perder os trinta metros de resolução.

## ESTRATEGIA
Vetorizar os dados anuais de area queimada e assim atribuir um ID para cada poligono fechado que represente um conjunto de pixeis conectados.

##ETAPAS:
#### ETAPA 0
Exportar dados do MapBiomas-Fogo coleção 2 para o Google Drive

script: [Download das imagens anuais de area queimada.js](https://code.earthengine.google.com/db5e8b901c15c5625fe7323b0e5be723?noload=1)


#### ETAPA 1
Vetorizar cicatrizes e enviar de volta ao Earth Engine como uma feature collection por ano.

PASSO 0: Mosaicar as imagens do ano em um só tif usando **gdalbuildvrt** e **gdal_translate**

PASSO 1: Vetorizar ela com **gdal_poligonize.py**

PASSO 2: Atribuir uma coluna de ID decimal a cada poligono com **GeoPandas**

PASSO 3: Enviar de volta ao Earth Engine como assets

NOTEBOOK: [Vetorização das cicatrizes.ipynb](https://colab.research.google.com/drive/1sgkU3j8s4UuYpY63Mfggnx6PSnM5dqsU?usp=sharing)


#### ETAPA 2: 
Rasterizar os vetores de cicatrizes anuais com a função **ee.Image().paint()** e construir um raster stack, de uma banda por ano com o valor do pixel sendo um id unico naquele ano para cada cicatriz

script: [rasterização dos dados de identificação de cicatrizes.js](https://code.earthengine.google.com/b4c08a9601167fe6eb381d909b73736f)

#### Outros resultados:
<picture>
  <img src="https://drive.google.com/uc?id=17fqkMS7klCCDgKWx2_YqTpeoCHx2q5r2">
</picture>
Base de dados: [Google Drive](https://drive.google.com/drive/folders/1zsbCJa88UyYBokRKCrgEA_cMXI82n22K?usp=sharing)

~~~javascript
// raster stack de indentificação das cicatrizes da coleção 2 do mapbiomas-fogo (1985-2022)
var scar_id = ee.Image('projects/mapbiomas-workspace/FOGO_COL2/SUBPRODUTOS/mapbiomas-fire-collection2-annual-burned-id-v1')
// raster stack com a area de cada cicatriz da coleção 2 do mapbiomas-fogo (1985-2022)
var scar_area = ee.Image('projects/mapbiomas-workspace/FOGO_COL2/SUBPRODUTOS/mapbiomas-fire-collection2-annual-burned-area-v1')
  
~~~
~~~javascript
// raster stack de indentificação das cicatrizes da coleção 1 do mapbiomas-fogo (1985-2020)
var scar_id = ee.Image('projects/workspace-ipam/assets/FOGO/mapbiomas-fire-collection1-annual-scar_id-1')
~~~
~~~javascript
// ASSETS: vetores das cicatrizes da coleção 2 do mapbiomas-fogo (1985 à 2022)
var scar_vector_1985 = ee.FeatureCollection('projects/workspace-ipam/assets/FOGO/VECTOR-COL2/mbfogo-col2-1985-v1');
var scar_vector_1986 = ee.FeatureCollection('projects/workspace-ipam/assets/FOGO/VECTOR-COL2/mbfogo-col2-1986-v1');
...
var scar_vector_2019 = ee.FeatureCollection('projects/workspace-ipam/assets/FOGO/VECTOR-COL2/mbfogo-col1-2021-v1');
var scar_vector_2020 = ee.FeatureCollection('projects/workspace-ipam/assets/FOGO/VECTOR-COL2/mbfogo-col1-2022-v1');

~~~
~~~javascript
// ASSETS: vetores das cicatrizes da coleção 1 do mapbiomas-fogo (1985 à 2020)
var scar_vector_1985 = ee.FeatureCollection('projects/workspace-ipam/assets/FOGO/VECTOR-COL1/mbfogo-col1-1985-v2');
var scar_vector_1986 = ee.FeatureCollection('projects/workspace-ipam/assets/FOGO/VECTOR-COL1/mbfogo-col1-1986-v2');
...
var scar_vector_2019 = ee.FeatureCollection('projects/workspace-ipam/assets/FOGO/VECTOR-COL1/mbfogo-col1-2019-v2');
var scar_vector_2020 = ee.FeatureCollection('projects/workspace-ipam/assets/FOGO/VECTOR-COL1/mbfogo-col1-2020-v2');

~~~

# DESENVOLVIMENTO DA ETAPA 1
Vetorizar cicatrizes e enviar de volta ao Earth Engine como uma feature collection por ano.

In [ ]:
# preparando o ambiente para o gdal
!apt-get install -y libgdal-dev
!apt-get install -y gdal-bin
!apt-get install python3-gdal

import os
os.environ['PROJ_LIB'] = '/usr/share/proj'
os.environ['GDAL_DATA'] = '/usr/share/gdal/2.2'
os.environ['GDAL_LIBRARY_PATH'] = '/usr/lib/libgdal.so'

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libgdal-dev is already the newest version (3.3.2+dfsg-2~focal2).
0 upgraded, 0 newly installed, 0 to remove and 22 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
gdal-bin is already the newest version (3.3.2+dfsg-2~focal2).
0 upgraded, 0 newly installed, 0 to remove and 22 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
python3-gdal is already the newest version (3.3.2+dfsg-2~focal2).
0 upgraded, 0 newly installed, 0 to remove and 22 not upgraded.


In [ ]:
# preparando ambiente para o geopandas
!pip install geopandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# conectando ao google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
PROJECT = 'mapbiomas'

from google.colab import auth
auth.authenticate_user()

# https://cloud.google.com/resource-manager/docs/creating-managing-projects

!gcloud config set project {PROJECT}

Updated property [core/project].


In [ ]:
import ee
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=bsfrkJ6uTZMdtLYSycE24Rq_mVZ8uWtraB9GfV5WuMs&tc=oQwPjKCuMsnMxj-ZE1Wfl990k9zkyNprABGn0JXoacE&cc=menbIe600wZrfh2_cHXuqNsxPuPAck_LvJQmS76Jqag

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AWtgzh6VLnfIbo2I01xG20VYTFUtz6_DE-mueFeNXgnGejtcwZkRVlROAnQ

Successfully saved authorization token.


In [ ]:
# importando as livrarias
# import os
# import ee
from osgeo import gdal
import geopandas as gpd
import numpy as np

In [ ]:
# endereço das imagens no Google Drive
home = '/content/drive/MyDrive/mapbiomas-fogo/mapbiomas_fire_collection2'

years = [
    # 1985,1986,1987,1988,1989,
      1990,1991,1992,1993,1994,
      1995,1996,1997,1998,1999,
    # 2000,2001,2002,2003,2004,
    # 2005,2006,2007,2008,2009,
    # 2010,2011,2012,2013,2014,
    # 2015,2016,2017,2018,2019,
    # 2020,2021,2022
]

all_files_tif = os.listdir(home)
all_files_tif = list(filter(lambda f: f.find('.tif') >= 0, all_files_tif))

for year in years:
  # year = 1985
  # PASSO 0: Mosaicar as imagens do ano em um só tif usando gdalbuildvrt e gdal_translate
  # aproximadamente 15 minutos por ano
  print('{0} {1}/{2} PASSO 0/3: Mosaicar as imagens do ano em um só tif usando gdalbuildvrt e gdal_translate '.format(year,years.index(year),len(years) - 1))

  filtered_images_tif = list(filter(lambda f: f.find('coverage_' + str(year)) >= 0, all_files_tif))
  for i, file_name in enumerate(filtered_images_tif):
    filtered_images_tif[i] = os.path.join(home, file_name)
  files_tif_str = " ".join(filtered_images_tif)
  # print(files_tif_str)
  name_out_tif = '{0}/mosaicos_anuais/mapbiomas_fire_collection2_burned_coverage_{1}-v1.tif'.format(home,year)
  name_out_vrt = '{0}/mosaicos_anuais/mapbiomas_fire_collection2_burned_coverage_{1}-v1.vrt'.format(home,year)

  # # rapido para muitas imagens ---- > compress 
  # os.system('gdalbuildvrt {0} {1}'.format(name_out_vrt, files_tif_str))
  # os.system('gdal_translate  -a_nodata 0 -co compress=DEFLATE {0} {1}'.format(name_out_vrt, name_out_tif))
  !gdalbuildvrt $name_out_vrt $files_tif_str
  !gdal_translate  -a_nodata 0 -co compress=DEFLATE $name_out_vrt $name_out_tif


  # PASSO 1: Vetorizar ela com gdal_poligonize.py
  file_name_shp = 'mapbiomas_fire_collection2_burned_coverage_{0}-v1.shp'.format(year)
  home_shp = '{0}/shapes_anuais/{1}'.format(home,file_name_shp)
  print('{0} {1}/{2} PASSO 1/3: Vetorizar ela com gdal_poligonize.py'.format(year,years.index(year),len(years) - 1))

  # os.system('gdal_polygonize.py {0} -b 1 -f "ESRI Shapefile" {1}'.format(name_out_tif,home_shp))
  !gdal_polygonize.py -b 1 $name_out_tif $home_shp

  # PASSO 2: Atribuir uma coluna de ID decimal a cada poligono com GeoPandas
  # || aproximadamente 9 minutos por ano
  print('{0} {1}/{2} PASSO 2/3: Atribuir uma coluna de ID decimal a cada poligono com GeoPandas'.format(year,years.index(year),len(years) - 1))



  gdf = gpd.read_file(home_shp)
  # ID decimal de identificação da cicatriz
  gdf['id'] = range(1, len(gdf) + 1)
  
  # # EXEMPLO DE ESTIMATIVAS QUE PODEM SER CALCULADAS NA ESCALA DA CICATRIZ UTILIZANDO DADOS VETORIAIS
  # # Cálculo da área
  # # Área (A): medida da extensão da superfície do polígono. Fórmula: A = area do polígono.
  # gdf['area'] = gdf.geometry.area

  # # Cálculo do perímetro
  # # Perímetro (P): medida do comprimento do contorno do polígono. Fórmula: P = comprimento do contorno do polígono.
  # gdf['perimetro'] = gdf.geometry.length

  # # Cálculo do índice de circularidade
  # # Índice de circularidade (IC): medida da semelhança do polígono com um círculo. Quanto mais próximo de 1, mais circular é o polígono. Fórmula: IC = (4 * pi * A) / (P ** 2)
  # gdf['ic'] = (4 * np.pi * gdf['area']) / (gdf['perimetro'] ** 2)

  # # Cálculo do índice de forma
  # # Índice de forma (IF): medida da complexidade do polígono. Quanto mais próximo de 0, mais simples é o polígono. Fórmula: IF = (P ** 2) / (4 * pi * A)
  # gdf['if'] = (gdf['perimetro'] ** 2) / (4 * np.pi * gdf['area'])

  new_shp = home_shp.replace("shapes_anuais","shapes_anuais_final")

  gdf.to_file(filename=new_shp, driver='ESRI Shapefile')
  
  # PASSO 3: Enviar de volta ao Earth Engine como assets
  print('{0} {1}/{2} PASSO 3/3: Enviar de volta ao Earth Engine como assets.'.format(year,years.index(year),len(years) - 1))

  os.system('gsutil cp {0} gs://shared-development-storage/AUXILIARES/MAPBIOMAS-FOGO/{1}'.format(new_shp,file_name_shp))
  os.system('gsutil cp {0} gs://shared-development-storage/AUXILIARES/MAPBIOMAS-FOGO/{1}'.format(new_shp.replace('.shp','.dbf'),file_name_shp.replace('.shp','.dbf')))
  os.system('gsutil cp {0} gs://shared-development-storage/AUXILIARES/MAPBIOMAS-FOGO/{1}'.format(new_shp.replace('.shp','.shx'),file_name_shp.replace('.shp','.shx')))
  os.system('gsutil cp {0} gs://shared-development-storage/AUXILIARES/MAPBIOMAS-FOGO/{1}'.format(new_shp.replace('.shp','.prj'),file_name_shp.replace('.shp','.prj')))

  asset_id = 'projects/workspace-ipam/assets/FOGO/VECTOR-COL2/mbfogo-col2-'+str(year)+'-v1'

  os.system('earthengine upload table --asset_id={0} gs://shared-development-storage/AUXILIARES/MAPBIOMAS-FOGO/{1}'.format(asset_id, filename_shp))

  print('{0} CONCLUIDO 🎉 {1}/{2}'.format(year,years.index(year),len(years) - 1))

print('concluido')

ERROR:fiona._env:PROJ: proj_identify: /usr/share/proj/proj.db contains DATABASE.LAYOUT.VERSION.MINOR = 0 whereas a number >= 2 is expected. It comes from another PROJ installation.
ERROR:fiona._env:PROJ: proj_identify: /usr/share/proj/proj.db contains DATABASE.LAYOUT.VERSION.MINOR = 0 whereas a number >= 2 is expected. It comes from another PROJ installation.


1990 0/9 PASSO 0/3: Mosaicar as imagens do ano em um só tif usando gdalbuildvrt e gdal_translate 
1990 0/9 PASSO 1/3: Vetorizar ela com gdal_poligonize.py
1990 0/9 PASSO 2/3: Atribuir uma coluna de ID decimal a cada poligono com GeoPandas


<ipython-input-11-4e8f9f86f711>:60: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf['area'] = gdf.geometry.area
<ipython-input-11-4e8f9f86f711>:64: UserWarning: Geometry is in a geographic CRS. Results from 'length' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf['perimetro'] = gdf.geometry.length
/usr/local/lib/python3.9/dist-packages/geopandas/io/file.py:545: UserWarning: You are attempting to write an empty DataFrame to file. For some drivers, this operation may fail.
  _to_file_fiona(df, filename, driver, schema, crs, mode, **kwargs)


1990 0/9 PASSO 3/3: Enviar de volta ao Earth Engine como assets.


ERROR:fiona._env:PROJ: proj_identify: /usr/share/proj/proj.db contains DATABASE.LAYOUT.VERSION.MINOR = 0 whereas a number >= 2 is expected. It comes from another PROJ installation.
ERROR:fiona._env:PROJ: proj_identify: /usr/share/proj/proj.db contains DATABASE.LAYOUT.VERSION.MINOR = 0 whereas a number >= 2 is expected. It comes from another PROJ installation.


...

1998 8/9 PASSO 3/3: Enviar de volta ao Earth Engine como assets.


ERROR:fiona._env:PROJ: proj_identify: /usr/share/proj/proj.db contains DATABASE.LAYOUT.VERSION.MINOR = 0 whereas a number >= 2 is expected. It comes from another PROJ installation.
ERROR:fiona._env:PROJ: proj_identify: /usr/share/proj/proj.db contains DATABASE.LAYOUT.VERSION.MINOR = 0 whereas a number >= 2 is expected. It comes from another PROJ installation.


1998 CONCLUIDO 🎉 8/9
1999 9/9 PASSO 0/3: Mosaicar as imagens do ano em um só tif usando gdalbuildvrt e gdal_translate 
1999 9/9 PASSO 1/3: Vetorizar ela com gdal_poligonize.py
1999 9/9 PASSO 2/3: Atribuir uma coluna de ID decimal a cada poligono com GeoPandas


<ipython-input-11-4e8f9f86f711>:60: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf['area'] = gdf.geometry.area
<ipython-input-11-4e8f9f86f711>:64: UserWarning: Geometry is in a geographic CRS. Results from 'length' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf['perimetro'] = gdf.geometry.length


1999 9/9 PASSO 3/3: Enviar de volta ao Earth Engine como assets.
1999 CONCLUIDO 🎉 9/9
concluido
